In [ ]:
from postprocessing import *
import scipy as scp
import glob
import os
from natsort import natsorted
from multiprocessing import Pool
import tqdm
try:
    from plotyy import * 
except:
    print('Cannot import plotyy')

In [ ]:
mpl.rcParams['figure.dpi']= 200

In [ ]:
import matplotlib.colors
color1 = "gold"
color2 = "darkmagenta"
color3 = "darkblue"
cmap_2 = matplotlib.colors.LinearSegmentedColormap.from_list("", [color1, color2])
cmap_3 = matplotlib.colors.LinearSegmentedColormap.from_list("", [color1, color2, color3])

In [ ]:
import scienceplots
plt.style.use(['science','no-latex'])
plt.rc('xtick', labelsize=15)    # fontsize of the tick labels
plt.rc('ytick', labelsize=15) 
plt.rcParams['axes.linewidth'] = 1.8 #set the value globally
plt.rcParams["font.family"] = "serif"
mpl.rcParams["axes.unicode_minus"] = False
plt.rcParams['xtick.major.size'] = 7
plt.rcParams['xtick.minor.size'] = 3

In [ ]:
def weighted_percentile(data, weights, perc):
    """
    perc : percentile in [0-1]!
    """
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(perc, cdf, data)

In [ ]:
def percentile_distance(storage, boxsize, density, machnumber, jetpower, start=15):
    """
    Create a table with jet tracer volume fraction and central density evolution with time
    Input: density    (number density of the simulation)
           machnumber (mach number of the simulation)
           jetpower   (jet power of the simulation)
           start      (initial snapshot from the turbulent box)
           a
    Output: array with three columns: time un Myr, jet tracer volume fraction, and mean number density within central 50pc
    
    """    
    simulation_directory = str(f'/n/{storage}/LABS/hernquist_lab/Users/borodina/{boxsize}/turb_jet_d{density}_m{machnumber}/jet{jetpower}_{start}')
    output_directory = simulation_directory + "/output/"
    figures_directory = simulation_directory + "/output/figures/"
    
    threshold_tracer = 1e-3
    i_file = 12
    distance_array = []
    time = 0
    if type(start) == str: start = 15
    
    while True and time < start + 5: #True or a number for a set length
        i_file += 1
        filename = "snap_%03d.hdf5" % (i_file)
        try:
            snap_data = h5py.File(output_directory + filename, "r+")
        except:
            try:
                i_file += 1
                filename = "snap_%03d.hdf5" % (i_file)
                snap_data = h5py.File(output_directory + filename, "r+")
            except:
                break
        center = 1000
        x = snap_data['PartType0/Coordinates'][:, 0] - center
        y = snap_data['PartType0/Coordinates'][:, 1] - center
        z = snap_data['PartType0/Coordinates'][:, 2] - center
        jet_tracer = snap_data['PartType0/Jet_Tracer'][:]
        mass = snap_data['PartType0/Masses'][:]

        mask_jet = jet_tracer > 1e-3
        time = get_time_from_snap(snap_data) * unit_time_in_megayr
        distance_sq = x ** 2 + y ** 2 + z ** 2

        if len(distance_sq[mask_jet]) > 0:
#             distance_100 = np.sqrt(np.percentile(distance_sq[mask_jet], 100))
#             distance_80 = np.sqrt(np.percentile(distance_sq[mask_jet], 80))
#             distance_50 = np.sqrt(np.percentile(distance_sq[mask_jet], 50))
            
            distance_100 = np.sqrt(weighted_percentile(distance_sq[mask_jet], jet_tracer[mask_jet] * mass[mask_jet], 1))
            distance_80 = np.sqrt(weighted_percentile(distance_sq[mask_jet], jet_tracer[mask_jet] * mass[mask_jet], 0.8))
            distance_50 = np.sqrt(weighted_percentile(distance_sq[mask_jet], jet_tracer[mask_jet] * mass[mask_jet], 0.5))
        else:
            distance_100 = 0
            distance_80 = 0
            distance_50 = 0

        distance_array.append([time, distance_100, distance_80, distance_50])
    return np.array(distance_array).T

In [ ]:
def x_loop(i):
    dist = (x - center - x_axis[i]) ** 2 + (y - center) ** 2 + (z - center) ** 2
    index = (dist < 30) & (x < x_axis[i] - dx) & (x > x_axis[i] + dx)  
    
    if (index).sum() > 0:
        return np.mean(ram_pressure[index]), np.mean(jet_tracer[index])
    else:
        index = np.argmin(dist)    
        return ram_pressure[index], jet_tracer[index]
    
def calculate_ram_pressure(filename):
    snap_data = h5py.File(output_directory + filename, "r")
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    density = snap_data['PartType0/Density'][:]

    center = 0.5 * snap_data["Header"].attrs["BoxSize"]
    v_total_sq = vx ** 2 + vy ** 2 + vz ** 2

    ram_pressure = density * v_total_sq * unit_density * unit_velocity * unit_velocity
    return ram_pressure

In [ ]:
def jet_and_density(storage, boxsize, density, machnumber, jetpower, start=15):
    """
    Create a table with jet tracer volume fraction and central density evolution with time
    Input: density    (number density of the simulation)
           machnumber (mach number of the simulation)
           jetpower   (jet power of the simulation)
           start      (initial snapshot from the turbulent box)
           
    Output: array with three columns: time un Myr, jet tracer volume fraction, and mean number density within central 50pc
    
    """    
    simulation_directory_jet = str(f'/n/{storage}/LABS/hernquist_lab/Users/borodina/{boxsize}/turb_jet_d{density}_m{machnumber}/jet{jetpower}_{start}')
    output_directory_jet = simulation_directory_jet + "/output/"
    figures_directory_jet = simulation_directory_jet + "/output/figures/"
    
    threshold_tracer = 1e-3
    result = []
    i_file = 12
    time = start
    while True and time < start + 5: #True or a number for a set length
        i_file += 1
        output_directory_jet = simulation_directory_jet + "/output/"
        filename = "snap_%03d.hdf5" % (i_file)
        try:
            snap_data = h5py.File(output_directory_jet + filename, "r+")
        except:
            try:
                i_file += 1
                filename = "snap_%03d.hdf5" % (i_file)
                snap_data = h5py.File(output_directory_jet + filename, "r+")
            except:
                break
        time = get_time_from_snap(snap_data) * unit_time_in_megayr
        x, y, z = snap_data['PartType0/Coordinates'][:].T
        center = snap_data['Header'].attrs['BoxSize'] / 2
        mask = (np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2) < 500)
        
        volume = snap_data['PartType0/Masses'][:][mask]/snap_data['PartType0/Density'][:][mask]
        tracer = snap_data['PartType0/Jet_Tracer'][:][mask]
        jet_volume = volume[tracer > threshold_tracer]
        jet_volume_fraction = np.sum(jet_volume)/np.sum(volume)
        mask = (np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2) < 50)
        n_mean = np.mean(snap_data['PartType0/Density'][mask]) * rho_to_numdensity
        result.append([time, jet_volume_fraction, n_mean])
    return result

In [ ]:
def calculate_warm_and_hot_outflows(output_directory, t_start):
    
    velocities_hot = []
    velocities_warm = []
    i_file = 12 - 1
    while True:
        i_file += 1
    
        projection = 'radial'
        try:
            filename = "snap_%03d.hdf5" % (i_file)
            snap_data = h5py.File(output_directory + filename, "r")
            time = get_time_from_snap(snap_data) * unit_time_in_megayr
            if time < t_start:
                continue
            if time > t_start + 1:
                break
        except:
            break
        
        x, y, z = snap_data['PartType0/Coordinates'][:].T
        vx, vy, vz = snap_data['PartType0/Velocities'][:].T
        center = snap_data['Header'].attrs['BoxSize'] / 2
        v_r = (vx * (x - center) + vy * (y - center) + vz * (z - center)) / \
                    np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)
    
        temperatures = get_temp(output_directory + filename, 5/3)
        masses = snap_data['PartType0/Masses'][:]
        densities = snap_data['PartType0/Density'][:]
        volumes = masses / densities
        radius = np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)
    
        mask_hot  = ((temperatures > 10 ** 4.5) & (radius > 400) & (radius < 500))
        mask_warm = ((temperatures < 10 ** 4.5) & (radius > 400) & (radius < 500))
    
        velocities_hot.append(np.histogram((v_r[mask_hot]), bins=bins, weights=masses[mask_hot] / bins_step)[0])
        velocities_warm.append(np.histogram((v_r[mask_warm]), bins=bins, weights=masses[mask_warm] / bins_step)[0])
    
    velocities_average_hot = np.mean(velocities_hot, axis=0)
    velocities_average_warm = np.mean(velocities_warm, axis=0)
    
    return velocities_average_hot, velocities_average_warm

## plot 1

In [ ]:
density = 20
mach = 4
jetpower = 38
start = '15'

In [ ]:
simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
################## 1
i_file = 0
filename = "snap_%03d.hdf5" % (i_file)
snap_data = h5py.File(output_directory +  "snap_%03d.hdf5" % (i_file),'r')

time_1 = get_time_from_snap(snap_data)
masses_1 = snap_data['PartType0/Masses'][:]
temperatures_1 = get_temp(output_directory + filename, 5/3, 'local')
density_1 = snap_data['PartType0/Density'][:] * rho_to_numdensity


################## 2
i_file = 3
filename = "snap_%03d.hdf5" % (i_file)
snap_data = h5py.File(output_directory +  "snap_%03d.hdf5" % (i_file),'r')

time_2 = get_time_from_snap(snap_data)
masses_2 = snap_data['PartType0/Masses'][:]
temperatures_2 = get_temp(output_directory + filename, 5/3, 'local')
density_2 = snap_data['PartType0/Density'][:] * rho_to_numdensity


################## 12
i_file = 12
filename = "snap_%03d.hdf5" % (i_file)
snap_data = h5py.File(output_directory +  "snap_%03d.hdf5" % (i_file),'r')

time_12 = get_time_from_snap(snap_data)
masses_12 = snap_data['PartType0/Masses'][:]
densities_12 = snap_data['PartType0/Density'][:]
temperatures_12 = get_temp(output_directory + filename, 5/3, 'local')
density_12 = snap_data['PartType0/Density'][:] * rho_to_numdensity


In [ ]:
t_bins = np.linspace(3.3, 6.5, 81)
n_bins = np.linspace(-3, 6, 81)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(6, 6))

axes[0].hist(np.log10(temperatures_1), weights=masses_1, density=False, 
         bins=t_bins,linewidth=2., color='orange', label="t=%.2f Myr"%(time_1 * unit_time_in_megayr), histtype='step')
axes[0].hist(np.log10(temperatures_2), weights=masses_2, density=False, 
         bins=t_bins, linewidth=2., color=color2, label="t=%.2f Myr"%(time_2 * unit_time_in_megayr), histtype='step')
axes[0].hist(np.log10(temperatures_12), weights=masses_12, density=False, 
         bins=t_bins, linewidth=2., color=color3, label="t=%.2f Myr"%(time_12 * unit_time_in_megayr), histtype='step')
axes[0].axvline(4.4, ls='--', c='black', label='log T = 4.4')
axes[0].legend(fontsize=12)
#plt.xlim(t_bins.min(), )# t_bins.max()
axes[0].set_ylim(1e1, 1e10)

axes[0].grid(ls='--', c='gray', alpha=0.2, zorder=100)
axes[0].set_yscale('log')
axes[0].set_xlabel("log T [K]", fontsize=15)
axes[0].set_ylabel(r"Mass [$M_\odot$]", fontsize=15)


axes[1].hist(np.log10(density_1), weights=masses_1, density=False, bins=n_bins,
             linewidth=2., color='orange', label="t=%.2f Myr"%(time_1 * unit_time_in_megayr), histtype='step')
axes[1].hist(np.log10(density_2), weights=masses_2, density=False, bins=n_bins,
             linewidth=2., color=color2, label="t=%.2f Myr"%(time_2 * unit_time_in_megayr), histtype='step')
axes[1].hist(np.log10(density_12), weights=masses_12, density=False, bins=n_bins,
             linewidth=2., color=color3, label="t=%.2f Myr"%(time_12 * unit_time_in_megayr), histtype='step')
#axes[1].axvline(4.45, c='black', label='log T = 4.45')
#plt.xlim(t_bins.min(), )# t_bins.max()
axes[1].set_ylim(1e1, 1e10)
axes[1].set_xlim(-3, 5)

axes[1].grid(ls='--', c='gray', alpha=0.2, zorder=100)
axes[1].set_yscale('log')
axes[1].set_xlabel(r"log n [cm$^{-3}$]", fontsize=15)
axes[1].set_ylabel(r"Mass [$M_\odot$]", fontsize=15)

fig.tight_layout()

## plot2, 3, 4

In [ ]:
density = 20
mach = 4
jetpower = 38
start = '15'

simulation_directory_jet = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

i_file = 50
filename = "snap_%03d.hdf5" % (i_file)
snap = snapshot(output_directory_jet + filename)
snap.project_field_paper(lbox=1000, slab_width=100, orientation='xy', show=False, cmap_jet=cmap_2, showbar=True, title=r'jet power $10^{38}$ erg/s')

In [ ]:
density = 20
mach = 4
jetpower = 40
start = '15'

simulation_directory_jet = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

i_file = 50
filename = "snap_%03d.hdf5" % (i_file)
snap = snapshot(output_directory_jet + filename)
snap.project_field_paper(lbox=1000, slab_width=100, orientation='xy', show=False, cmap_jet=cmap_2, showbar=False,  title=r'jet power $10^{40}$ erg/s')

In [ ]:
density = 20
mach = 4
jetpower = 43
start = '15'

simulation_directory_jet = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

i_file = 20
filename = "snap_%03d.hdf5" % (i_file)
snap = snapshot(output_directory_jet + filename)
snap.project_field_paper(lbox=1000, slab_width=100, orientation='xy', show=False, cmap_jet=cmap_2, showbar=False,  title=r'jet power $10^{43}$ erg/s')

## plot next

In [ ]:
dist_38_15 = percentile_distance('holylfs05', '2kpc', 20, 4, 38, '15')
dist_40_15 = percentile_distance('holylfs05', '2kpc', 20, 4, 40, '15')
dist_43_15 = percentile_distance('holylfs05', '2kpc', 20, 4, 43, '15')

In [ ]:
dist_43_uniform  = percentile_distance('holylfs05', '2kpc', 20, 8, 43, 'uniform')
dist_40_uniform  = percentile_distance('holylfs05', '2kpc', 20, 8, 40, 'uniform')
dist_38_uniform  = percentile_distance('holylfs05', '2kpc', 20, 8, 38, 'uniform')

In [ ]:
dist_43_uniform_dilute  = percentile_distance('holylfs05', '2kpc', 20, 8, 43, 'uniform_dilute')
dist_40_uniform_dilute  = percentile_distance('holylfs05', '2kpc', 20, 8, 40, 'uniform_dilute')
dist_38_uniform_dilute  = percentile_distance('holylfs05', '2kpc', 20, 8, 38, 'uniform_dilute')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10,3), sharey=True)


# blues = mpl.cm.get_cmap("Blues", 10) blues(5,7,9)
c1 = 'crimson'#cmap_2(255)
c2 = 'darkred'#cmap_2(180)
c3 = 'lightsalmon'#cmap_2(110)


# axes[0].plot(dist_43_15[0][:15], dist_43_15[2][:15], c='tab:red', ls='-', ) 'tab:red', 'darkred', 'lightcoral'
# axes[1].plot(dist_40_15[0], dist_40_15[2], c='mediumblue', ls='-', )        'mediumblue', 'darkblue', 'tab:blue'
# axes[2].plot(dist_38_15[0], dist_38_15[2], c='forestgreen', ls='-', )       'forestgreen', 'darkgreen', 'tab:green'

#for legend
axes[0].fill_between(-dist_43_15[0][:15], -dist_43_15[1][:15], -dist_43_15[3][:15], color=c1, ls='-', alpha=0.4, label='turbulent gas')
axes[0].fill_between(-dist_43_uniform_dilute[0][:15], -dist_43_uniform_dilute[1][:15], -dist_43_uniform_dilute[3][:15], color=c3, ls='-', alpha=0.4, hatch='///', label='uniform dilute gas')
axes[0].fill_between(-dist_43_uniform[0], -dist_43_uniform[1], -dist_43_uniform[3], color=c2, ls='-', alpha=0.4, hatch='\\\///', label='uniform gas')

#start plotting here
axes[0].plot(dist_43_15[0][:15], dist_43_15[2][:15], c=c1, ls='-', lw=2, zorder=100)
axes[1].plot(dist_40_15[0], dist_40_15[2], c=c1, ls='-', lw=2, zorder=100)
axes[2].plot(dist_38_15[0], dist_38_15[2], c=c1, ls='-', lw=2, zorder=100)

axes[0].fill_between(dist_43_15[0][:15], dist_43_15[1][:15], dist_43_15[3][:15], color=c1, ls='-', alpha=0.4, zorder=100)
axes[1].fill_between(dist_40_15[0], dist_40_15[1], dist_40_15[3], color=c1, ls='-', alpha=0.4, zorder=100)
axes[2].fill_between(dist_38_15[0], dist_38_15[1], dist_38_15[3], color=c1, ls='-', alpha=0.4, zorder=100)

axes[0].plot(dist_43_uniform[0], dist_43_uniform[1], c=c2, ls='--', lw=2)
axes[1].plot(dist_40_uniform[0], dist_40_uniform[1], c=c2, ls='--', lw=2)
axes[2].plot(dist_38_uniform[0], dist_38_uniform[1], c=c2, ls='--', lw=2)

axes[0].fill_between(dist_43_uniform[0], dist_43_uniform[1], dist_43_uniform[3], color=c2, ls='-', alpha=0.4, hatch='\\\///')
axes[1].fill_between(dist_40_uniform[0], dist_40_uniform[1], dist_40_uniform[3], color=c2, ls='-', alpha=0.4, hatch='\\\///')
axes[2].fill_between(dist_38_uniform[0], dist_38_uniform[1], dist_38_uniform[3], color=c2, ls='-', alpha=0.4, hatch='\\\///')


axes[0].plot(dist_43_uniform_dilute[0][:15], dist_43_uniform_dilute[1][:15], c=c3, ls='--', lw=2)
axes[1].plot(dist_40_uniform_dilute[0][:35], dist_40_uniform_dilute[1][:35], c=c3, ls='--', lw=2)
axes[2].plot(dist_38_uniform_dilute[0], dist_38_uniform_dilute[1], c=c3, ls='--', lw=2)

axes[0].fill_between(dist_43_uniform_dilute[0][:15], dist_43_uniform_dilute[1][:15], dist_43_uniform_dilute[3][:15], color=c3, ls='-', alpha=0.4, hatch='///')
axes[1].fill_between(dist_40_uniform_dilute[0][:35], dist_40_uniform_dilute[1][:35], dist_40_uniform_dilute[3][:35], color=c3, ls='-', alpha=0.4, hatch='///')
axes[2].fill_between(dist_38_uniform_dilute[0], dist_38_uniform_dilute[1], dist_38_uniform_dilute[3], color=c3, ls='-', alpha=0.4, hatch='///')

for ax in axes:
    ax.set_xlabel('t [Myr]', fontsize=15)
    ax.set_xlim(15, 20)
    ax.set_ylim(0, 750)
    ax.grid(ls='--', c='gray', alpha=0.4, zorder=0)

axes[0].set_ylabel(r'$r_{80}$ [pc]', fontsize=15, zorder=100)

axes[0].set_title(r'jet power $10^{43}$ erg/s')
axes[1].set_title(r'jet power $10^{40}$ erg/s')
axes[2].set_title(r'jet power $10^{38}$ erg/s')

# axes[1].get_yaxis().set_ticklabels([])
# axes[2].get_yaxis().set_ticklabels([])

axes[0].legend()
# axes[0].legend(fontsize=6, loc='upper right', framealpha=0.6)
fig.tight_layout()

plt.show()

## plot next

In [ ]:
density = 20
mach = 4
jetpower = 40
start = '15'

simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
i_file = 13 # skip snap 0
ram_pressure_list_parallel = []
jet_tracer_fall_x_parallel_left = []
jet_tracer_fall_x_parallel_right = []
time_parallel = []

x_axis = np.linspace(-500, 500, 101)
mask_left = (x_axis < -30)
mask_right = (x_axis > 30)

dx = x_axis[1] - x_axis[0]

#for i_file in tqdm.trange(12, 42):
    
for i_file in tqdm.trange(13, 116):
    filename = "snap_%03d.hdf5" % (i_file)

    snap_data = h5py.File(output_directory + filename, "r")
    
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    density = snap_data['PartType0/Density'][:]
    ram_pressure = calculate_ram_pressure(filename)
    jet_tracer = snap_data['PartType0/Jet_Tracer'][:]
    center = 0.5 * snap_data["Header"].attrs["BoxSize"]

    pool = Pool(32)
    ram_pressure_axis = []
    jet_tracer_axis = []
    for ram_pressure_axis_1, jet_tracer_axis_1 in pool.map(x_loop, np.arange(len(x_axis)), chunksize=1):
        jet_tracer_axis.append(jet_tracer_axis_1)
        ram_pressure_axis.append(ram_pressure_axis_1)

    pool.close()
    jet_tracer_fall_x_parallel_left.append(x_axis[mask_left][np.array(jet_tracer_axis)[mask_left]<5e-2][-1])
    jet_tracer_fall_x_parallel_right.append(x_axis[mask_right][np.array(jet_tracer_axis)[mask_right]<5e-2][0])
    ram_pressure_list_parallel.append(np.array(ram_pressure_axis))

    time_parallel.append(get_time_from_snap(snap_data) * unit_time_in_megayr - 15)


In [ ]:
time = np.array(time_parallel)
real_y = np.flip(x_axis)
real_x = time + 15

dx = (real_x[1]-real_x[0])/2.
dy = (real_y[1]-real_y[0])/2.
extent = [real_x[0]-dx, real_x[-1]+dx, real_y[0]-dy, real_y[-1]+dy]


In [ ]:
time = np.array(time_parallel)
real_y = np.flip(x_axis)
real_x = time + 15

dx = (real_x[1]-real_x[0])/2.
dy = (real_y[1]-real_y[0])/2.
extent = [real_x[0]-dx, real_x[-1]+dx, real_y[0]-dy, real_y[-1]+dy]


fig, ax = plt.subplots(figsize=(5, 5))
im = plt.imshow(np.array(ram_pressure_list_parallel).T, cmap=cmap_3.reversed(), extent=extent, aspect='auto', norm=colors.LogNorm(vmin=1e-11, vmax=1e-7))#
#plt.colorbar(im, label=r'number density, cm$^{-3}$')
cb = plt.colorbar(im, label='ram pressure, Bar')
cb.set_label(label='ram pressure, Bar', fontsize=15)
# ax.set(xticks=np.arange(0, len(time), 10), xticklabels=np.arange(time.min(), time.max(), 10));

plt.plot(real_x, np.array(jet_tracer_fall_x_parallel_left),lw=2,  c='white', zorder=100)
plt.plot(real_x, np.array(jet_tracer_fall_x_parallel_right),lw=2,  c='white', zorder=100)

plt.gca().invert_yaxis()
plt.xlim(15, 20)
#plt.yticks(ticks=(-100, -200, -300, -400, -500), labels=(100, 200, 300, 400, 500))
plt.xlabel('time, Myr', fontsize=15)
plt.ylabel('x, pc', fontsize=15)

## plot outflows

In [ ]:
bins = np.linspace(-900, 900, 101)
bins_step = bins[1] - bins[0]

In [ ]:
density = '20'
mach    = '4'
jet     = '43'
start   = '15'
t_start = 15

simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jet}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

velocities_average_hot_43, velocities_average_warm_43 = calculate_warm_and_hot_outflows(output_directory, t_start)

simulation_directory_turb = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/turb')
output_directory_turb = simulation_directory_turb + "/output/"

velocities_average_hot_turb_43, velocities_average_warm_turb_43 = calculate_warm_and_hot_outflows(output_directory_turb, t_start)

In [ ]:
density = '20'
mach    = '4'
jet     = '40'
start   = '15'
t_start = 16.5

simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jet}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

velocities_average_hot_40, velocities_average_warm_40 = calculate_warm_and_hot_outflows(output_directory, t_start)

simulation_directory_turb = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/turb')
output_directory_turb = simulation_directory_turb + "/output/"

velocities_average_hot_turb_40, velocities_average_warm_turb_40 = calculate_warm_and_hot_outflows(output_directory_turb, t_start)

In [ ]:
density = '20'
mach    = '4'
jet     = '38'
start   = '15'
t_start = 18.5

simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jet}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

velocities_average_hot_38, velocities_average_warm_38 = calculate_warm_and_hot_outflows(output_directory, t_start)

simulation_directory_turb = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/turb')
output_directory_turb = simulation_directory_turb + "/output/"

velocities_average_hot_turb_38, velocities_average_warm_turb_38 = calculate_warm_and_hot_outflows(output_directory_turb, t_start)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(11,3), sharey=True)

axes[0].step(bins[:-1], velocities_average_warm_43, where='mid', label='warm gas with jet on', color='tab:blue', linewidth=1.5)
axes[0].step(bins[:-1], velocities_average_hot_43, where='mid', label='hot gas with jet on', color='tab:red', linewidth=1.5)

axes[0].fill_between(bins[:-1], velocities_average_warm_turb_43, step='mid', label='warm gas', color='tab:blue', alpha=0.5, edgecolor='none')
axes[0].fill_between(bins[:-1], velocities_average_hot_turb_43, step='mid', label='hot gas', color='tab:red', alpha=0.5, edgecolor='none')

axes[1].step(bins[:-1], velocities_average_warm_40, where='mid', label='warm gas with jet on', color='tab:blue', linewidth=1.5)
axes[1].step(bins[:-1], velocities_average_hot_40, where='mid', label='hot gas with jet on', color='tab:red', linewidth=1.5)

axes[1].fill_between(bins[:-1], velocities_average_warm_turb_40, step='mid', label='warm gas', color='tab:blue', alpha=0.5, edgecolor='none')
axes[1].fill_between(bins[:-1], velocities_average_hot_turb_40, step='mid', label='hot gas', color='tab:red', alpha=0.5, edgecolor='none')


axes[2].step(bins[:-1], velocities_average_warm_38, where='mid', label='warm gas with jet on', color='tab:blue', linewidth=1.5)
axes[2].step(bins[:-1], velocities_average_hot_38, where='mid', label='hot gas with jet on', color='tab:red', linewidth=1.5)

axes[2].fill_between(bins[:-1], velocities_average_warm_turb_38, step='mid', label='warm gas', color='tab:blue', alpha=0.5, edgecolor='none')
axes[2].fill_between(bins[:-1], velocities_average_hot_turb_38, step='mid', label='hot gas', color='tab:red', alpha=0.5, edgecolor='none')

for ax in axes:   
    ax.grid(ls='--', zorder = 100, alpha=0.5)
    ax.set_xlim(-600, 600)
    ax.set_ylim(1e-2, 1e6)
    ax.set_xlabel(r"$v_r$ [km / s]", fontsize=15)
    ax.set_yscale('log')


axes[0].set_title(r'jet power $10^{43}$ erg/s', fontsize=12)
axes[1].set_title(r'jet power $10^{40}$ erg/s', fontsize=12)
axes[2].set_title(r'jet power $10^{38}$ erg/s', fontsize=12)

axes[0].legend(loc='upper left',fontsize=6, fancybox=True, framealpha=0.5)
axes[0].set_ylabel(r'$\frac{M_\odot}{{km / s}}$ of cells with ' + '\n'+ r'$r_{cell} \in [400; 500]$ pc', fontsize=12)

fig.tight_layout()

## appendix

In [ ]:
# density = 10
# mach = 8
# jetpower = 40
# start = '15'

# simulation_directory_jet = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
# output_directory_jet = simulation_directory_jet + "/output/"
# figures_directory_jet = simulation_directory_jet + "/output/figures/"

In [ ]:
# i_file = 30
# filename = "snap_%03d.hdf5" % (i_file)
# snap = snapshot(output_directory_jet + filename)
# snap.project_field_paper(lbox=1000, slab_width=100, orientation='xy', show=False, cmap_jet=cmap_2)

# i_file = 35
# filename = "snap_%03d.hdf5" % (i_file)
# snap = snapshot(output_directory_jet + filename)
# snap.project_field_paper(lbox=1000, slab_width=100, orientation='xy', show=False, cmap_jet=cmap_2)


# i_file = 40
# filename = "snap_%03d.hdf5" % (i_file)
# snap = snapshot(output_directory_jet + filename)
# snap.project_field_paper(lbox=1000, slab_width=100, orientation='xy', show=False, cmap_jet=cmap_2)

# i_file = 45
# filename = "snap_%03d.hdf5" % (i_file)
# snap = snapshot(output_directory_jet + filename)
# snap.project_field_paper(lbox=1000, slab_width=100, orientation='xy', show=False, cmap_jet=cmap_2)

# # i_file = 50
# # filename = "snap_%03d.hdf5" % (i_file)
# # snap = snapshot(output_directory_jet + filename)
# # snap.project_field_paper(lbox=1000, slab_width=100, orientation='xy', show=False, cmap_jet=cmap_2)

## appendix

In [ ]:
density = 20
mach = 8
jetpower = 38
start = 'uniform_dilute'

simulation_directory_jet = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

i_file = 50
filename = "snap_%03d.hdf5" % (i_file)
snap = snapshot(output_directory_jet + filename)
snap.project_field_paper(lbox=1000, slab_width=100, orientation='xy', show=False, cmap_jet=cmap_2, showbar=True, title=r'jet power $10^{38}$ erg/s',vmin0=-5,vmax0=-1)

In [ ]:
density = 20
mach = 8
jetpower = 40
start = 'uniform_dilute'

simulation_directory_jet = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

i_file = 20
filename = "snap_%03d.hdf5" % (i_file)
snap = snapshot(output_directory_jet + filename)
snap.project_field_paper(lbox=1000, slab_width=100, orientation='xy', show=False, cmap_jet=cmap_2, showbar=False, title=r'jet power $10^{40}$ erg/s')

In [ ]:
density = 20
mach = 8
jetpower = 43
start = 'uniform_dilute'

simulation_directory_jet = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

i_file = 14
filename = "snap_%03d.hdf5" % (i_file)
snap = snapshot(output_directory_jet + filename)
snap.project_field_paper(lbox=1000, slab_width=100, orientation='xy', show=False, cmap_jet=cmap_2, showbar=False, title=r'jet power $10^{43}$ erg/s')

In [ ]:
density = 20
mach = 2
jetpower = 40
start = '25'

simulation_directory_jet = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

i_file = 85
filename = "snap_%03d.hdf5" % (i_file)
snap = snapshot(output_directory_jet + filename)
snap.project_field_paper(lbox=1000, slab_width=100, orientation='xy', show=False, cmap_jet=cmap_2, showbar=False, title=r'$\mathfrak{M} = 2$')

In [ ]:
density = 20
mach = 8
jetpower = 40
start = '15'

simulation_directory_jet = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

i_file = 50
filename = "snap_%03d.hdf5" % (i_file)
snap = snapshot(output_directory_jet + filename)
snap.project_field_paper(lbox=1000, slab_width=100, orientation='xy', show=False, cmap_jet=cmap_2, showbar=False, title=r'$\mathfrak{M} = 8$')

In [ ]:
density = 20
mach = 14
jetpower = 40
start = '15'

simulation_directory_jet = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

i_file = 50
filename = "snap_%03d.hdf5" % (i_file)
snap = snapshot(output_directory_jet + filename)
snap.project_field_paper(lbox=1000, slab_width=100, orientation='xy', show=False, cmap_jet=cmap_2, showbar=True, title=r'$\mathfrak{M} = 14$')

## jet tracer things for different mach numbers

In [ ]:
result_2  = np.array(jet_and_density('holylfs05', '2kpc', 20, 2, 40, 25)).T
result_4  = np.array(jet_and_density('holylfs05', '2kpc', 20, 4, 40, 15)).T
result_8  = np.array(jet_and_density('holylfs05', '2kpc', 20, 8, 40, 15)).T

In [ ]:
dist_2 = percentile_distance('holylfs05', '2kpc', 20, 2, 40, 25)
dist_4 = percentile_distance('holylfs05', '2kpc', 20, 4, 40, 15)
dist_8 = percentile_distance('holylfs05', '2kpc', 20, 8, 40, 15)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(9,3))

c2 = '#6B5B95'#'#d34830'#cmap_2(255)
c1 = '#FF6F61'#'#5ea836'#cmap_2(180)
c3 = '#88B04B'#'tab:blue'#cmap_2(110)


axes[0].plot(dist_2[0]-25, dist_2[2], label='M=2', c=c3, ls='-',lw=2, zorder=100)
axes[0].plot(dist_4[0]-15, dist_4[2], label='M=4', c=c1, ls='-',lw=2, zorder=100)
axes[0].plot(dist_8[0]-15, dist_8[2], label='M=8', c=c2, ls='-',lw=2, zorder=100)


axes[0].fill_between(dist_2[0]-25, dist_2[1], dist_2[3], color=c3, ls='-', alpha=0.5, zorder=90)
axes[0].fill_between(dist_4[0]-15, dist_4[1], dist_4[3], color=c1, ls='-', alpha=0.5)
axes[0].fill_between(dist_8[0]-15, dist_8[1], dist_8[3], color=c2, ls='-', alpha=0.5)



axes[0].set_xlabel('t [Myr]', fontsize=15)
axes[0].set_xlim(0, 5)
axes[0].set_ylim(0, 750)
axes[0].grid(ls='--', c='gray', alpha=0.4, zorder=0)
axes[0].set_ylabel(r'$r_{80}$ [pc]', fontsize=15, zorder=100)


axes[1].plot(result_2[0]-25, result_2[1], c=c3,  ls='-',lw=2)
axes[1].plot(result_4[0]-15, result_4[1], c=c1, ls='-',lw=2)
axes[1].plot(result_8[0]-15, result_8[1], c=c2, ls='-',lw=2)

axes[1].set_xlabel('t [Myr]', fontsize=15)
axes[1].set_xlim(0, 5)

axes[1].set_ylabel('jet tracer volume fraction', zorder=100, fontsize=15)
# axes[1].set_yscale('log')
axes[1].grid(ls='--', c='gray', alpha=0.4, zorder=0)
axes[0].legend(fontsize=11, loc='upper left', framealpha=0.6)
fig.tight_layout()

plt.show()

## plot ram pressure for mach number

In [ ]:
density = 20
mach = 2
jetpower = 40
start = '25'

simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
ram_pressure_list_parallel_2 = []
jet_tracer_fall_x_parallel_2_left = []
jet_tracer_fall_x_parallel_2_right = []
time_parallel_2 = []

x_axis = np.linspace(-500, 500, 101)
mask_left = (x_axis < -30)
mask_right = (x_axis > 30)

dx = x_axis[1] - x_axis[0]

#for i_file in tqdm.trange(12, 42):
    
for i_file in tqdm.trange(15, 130):
    filename = "snap_%03d.hdf5" % (i_file)

    snap_data = h5py.File(output_directory + filename, "r")
    
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    density = snap_data['PartType0/Density'][:]
    ram_pressure = calculate_ram_pressure(filename)
    jet_tracer = snap_data['PartType0/Jet_Tracer'][:]
    center = 0.5 * snap_data["Header"].attrs["BoxSize"]

    pool = Pool(32)
    ram_pressure_axis = []
    jet_tracer_axis = []
    for ram_pressure_axis_1, jet_tracer_axis_1 in pool.map(x_loop, np.arange(len(x_axis)), chunksize=1):
        jet_tracer_axis.append(jet_tracer_axis_1)
        ram_pressure_axis.append(ram_pressure_axis_1)

    pool.close()
    
    jet_tracer_fall_x_parallel_2_left.append(x_axis[mask_left][np.array(jet_tracer_axis)[mask_left]<5e-2][-1])
    jet_tracer_fall_x_parallel_2_right.append(x_axis[mask_right][np.array(jet_tracer_axis)[mask_right]<5e-2][0])
    ram_pressure_list_parallel_2.append(np.array(ram_pressure_axis))
    
    time_parallel_2.append(get_time_from_snap(snap_data) * unit_time_in_megayr - 25)


In [ ]:
density = 20
mach = 8
jetpower = 40
start = '15'

simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
ram_pressure_list_parallel_8 = []
jet_tracer_fall_x_parallel_8_left = []
jet_tracer_fall_x_parallel_8_right = []
time_parallel_8 = []

x_axis = np.linspace(-500, 500, 101)
mask_left = (x_axis < -30)
mask_right = (x_axis > 30)

dx = x_axis[1] - x_axis[0]

#for i_file in tqdm.trange(12, 42):
    
for i_file in tqdm.trange(13, 116):
    filename = "snap_%03d.hdf5" % (i_file)

    snap_data = h5py.File(output_directory + filename, "r")
    
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    density = snap_data['PartType0/Density'][:]
    ram_pressure = calculate_ram_pressure(filename)
    jet_tracer = snap_data['PartType0/Jet_Tracer'][:]
    center = 0.5 * snap_data["Header"].attrs["BoxSize"]

    
    pool = Pool(32)
    ram_pressure_axis = []
    jet_tracer_axis = []
    for ram_pressure_axis_1, jet_tracer_axis_1 in pool.map(x_loop, np.arange(len(x_axis)), chunksize=1):
        jet_tracer_axis.append(jet_tracer_axis_1)
        ram_pressure_axis.append(ram_pressure_axis_1)

    pool.close()
    
    jet_tracer_fall_x_parallel_8_left.append(x_axis[mask_left][np.array(jet_tracer_axis)[mask_left]<5e-2][-1])
    jet_tracer_fall_x_parallel_8_right.append(x_axis[mask_right][np.array(jet_tracer_axis)[mask_right]<5e-2][0])
    ram_pressure_list_parallel_8.append(np.array(ram_pressure_axis))
    
    time_parallel_8.append(get_time_from_snap(snap_data) * unit_time_in_megayr - 15)


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10.5, 5), sharey=True,gridspec_kw=dict(width_ratios=[1, 1.22]))

step_8 = time_parallel_8[1] - time_parallel_8[0]
step_2  = time_parallel_2[1]  - time_parallel_2[0]

extent_8 = [time_parallel_8[0] + 15 - step_8, time_parallel_8[-1] + 15 + step_8, x_axis[-1]+dx, x_axis[0]-dx]
extent_2 = [time_parallel_2[0] + 25 - step_2, time_parallel_2[-1] + 25 + step_2, x_axis[-1]+dx, x_axis[0]-dx]

im = axes[0].imshow(np.array(ram_pressure_list_parallel_2).T, cmap=cmap_3.reversed(), extent=extent_2, aspect='auto', norm=colors.LogNorm(vmin=1e-11, vmax=1e-7))#
axes[1].imshow(np.array(ram_pressure_list_parallel_8).T, cmap=cmap_3.reversed(), extent=extent_8, aspect='auto', norm=colors.LogNorm(vmin=1e-11, vmax=1e-7))#

cb = plt.colorbar(im, label='ram pressure, Bar')
cb.set_label(label='ram pressure, Bar', fontsize=15)

axes[0].plot(np.array(time_parallel_2) + 25,  np.array(jet_tracer_fall_x_parallel_2_left), lw=2, c='white', zorder=100)
axes[0].plot(np.array(time_parallel_2) + 25,  np.array(jet_tracer_fall_x_parallel_2_right), lw=2, c='white', zorder=100)
axes[1].plot(np.array(time_parallel_8) + 15, np.array(jet_tracer_fall_x_parallel_8_left), lw=2, c='white', zorder=100)
axes[1].plot(np.array(time_parallel_8) + 15, np.array(jet_tracer_fall_x_parallel_8_right), lw=2, c='white', zorder=100)


axes[0].set_xlim(25, 30)
# axes[1].set_xlim(15, 20)

axes[0].set_title(r'$\mathfrak{M} = 2$', fontsize=15)
axes[1].set_title(r'$\mathfrak{M} = 8$', fontsize=15)

axes[0].invert_yaxis()
# axes[1].invert_yaxis()
axes[1].set_xticks(ticks=np.arange(15, 21, 1))

axes[0].set_xlabel('time, Myr', fontsize=15)
axes[1].set_xlabel('time, Myr', fontsize=15)
axes[0].set_ylabel('x, pc', fontsize=15)

## outflows for mach

In [ ]:
bins = np.linspace(-900, 900, 101)
bins_step = bins[1] - bins[0]

In [ ]:
density = '20'
mach    = '2'
jet     = '40'
start   = '25'
t_start = 27.5

simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jet}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

velocities_average_hot_2, velocities_average_warm_2 = calculate_warm_and_hot_outflows(output_directory, t_start)

simulation_directory_turb = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/turb_{start}')
output_directory_turb = simulation_directory_turb + "/output/"

velocities_average_hot_turb_2, velocities_average_warm_turb_2 = calculate_warm_and_hot_outflows(output_directory_turb, t_start)

In [ ]:
density = '20'
mach    = '4'
jet     = '40'
start   = '15'
t_start = 16.5

simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jet}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

velocities_average_hot_4, velocities_average_warm_4 = calculate_warm_and_hot_outflows(output_directory, t_start)

simulation_directory_turb = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/turb')
output_directory_turb = simulation_directory_turb + "/output/"

velocities_average_hot_turb_4, velocities_average_warm_turb_4 = calculate_warm_and_hot_outflows(output_directory_turb, t_start)

In [ ]:
density = '20'
mach    = '8'
jet     = '40'
start   = '15'
t_start = 16.5

simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jet}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

velocities_average_hot_8, velocities_average_warm_8 = calculate_warm_and_hot_outflows(output_directory, t_start)

simulation_directory_turb = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/turb')
output_directory_turb = simulation_directory_turb + "/output/"

velocities_average_hot_turb_8, velocities_average_warm_turb_8 = calculate_warm_and_hot_outflows(output_directory_turb, t_start)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(11,3), sharey=True)

axes[0].step(bins[:-1], velocities_average_warm_2, where='mid', label='warm gas with jet on', color='tab:blue', linewidth=1.5)
axes[0].step(bins[:-1], velocities_average_hot_2, where='mid', label='hot gas with jet on', color='tab:red', linewidth=1.5)

axes[0].fill_between(bins[:-1], velocities_average_warm_turb_2, step='mid', label='warm gas', color='tab:blue', alpha=0.5, edgecolor='none')
axes[0].fill_between(bins[:-1], velocities_average_hot_turb_2, step='mid', label='hot gas', color='tab:red', alpha=0.5, edgecolor='none')

axes[1].step(bins[:-1], velocities_average_warm_4, where='mid', label='warm gas with jet on', color='tab:blue', linewidth=1.5)
axes[1].step(bins[:-1], velocities_average_hot_4, where='mid', label='hot gas with jet on', color='tab:red', linewidth=1.5)

axes[1].fill_between(bins[:-1], velocities_average_warm_turb_4, step='mid', label='warm gas', color='tab:blue', alpha=0.5, edgecolor='none')
axes[1].fill_between(bins[:-1], velocities_average_hot_turb_4, step='mid', label='hot gas', color='tab:red', alpha=0.5, edgecolor='none')


axes[2].step(bins[:-1], velocities_average_warm_8, where='mid', label='warm gas with jet on', color='tab:blue', linewidth=1.5)
axes[2].step(bins[:-1], velocities_average_hot_8, where='mid', label='hot gas with jet on', color='tab:red', linewidth=1.5)

axes[2].fill_between(bins[:-1], velocities_average_warm_turb_8, step='mid', label='warm gas', color='tab:blue', alpha=0.5, edgecolor='none')
axes[2].fill_between(bins[:-1], velocities_average_hot_turb_8, step='mid', label='hot gas', color='tab:red', alpha=0.5, edgecolor='none')

for ax in axes:   
    ax.grid(ls='--', zorder = 100, alpha=0.5)
    ax.set_xlim(-600, 600)
    ax.set_ylim(1e-2, 4e6)
    ax.set_xlabel(r"$v_r$ [km / s]", fontsize=15)
    ax.set_yscale('log')


axes[0].set_title(r'$\mathfrak{M} = 2$', fontsize=12)
axes[1].set_title(r'$\mathfrak{M} = 4$', fontsize=12)
axes[2].set_title(r'$\mathfrak{M} = 8$', fontsize=12)

axes[0].legend(loc='upper left',fontsize=6, fancybox=True, framealpha=0.5)
axes[0].set_ylabel(r'$\frac{M_\odot}{{km / s}}$ of cells with ' + '\n'+ r'$r_{cell} \in [400; 500]$ pc', fontsize=12)

fig.tight_layout()

### turb box maps

In [ ]:
density = 20
mach = 4

simulation_directory_jet = str( f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/turb')
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

In [ ]:
i_file = 13 # 21
snap = snapshot(output_directory_jet + "snap_%03d.hdf5" % (i_file))
snap.overlap_temp(lbox=1000, slab_width=100, orientation='xy', showbar=True, show=False)